In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
# Load API Credentials
with open('/Users/CHBAM/.secret/yelp_api.json') as f:
    login = json.load(f)
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [3]:
# set our API call parameters 
LOCATION = 'NY,NY'
TERM = 'Burger'

In [4]:
JSON_FILE = "Data/results_NY_Burger.json"
folder = os.path.dirname(JSON_FILE)
## If JSON_FILE included a folder:
if len(folder)>0:
   # create the folder
   os.makedirs(folder,exist_ok=True)
   
## INFORM USER AND SAVE EMPTY LIST
print(f'[i] {JSON_FILE} not found. Saving empty list to file.')


# save an empty list
with open(JSON_FILE,'w') as f:
   json.dump([],f)  

[i] Data/results_NY_Burger.json not found. Saving empty list to file.


In [7]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                                offset=0)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [8]:
total_results = results['total']
total_results

14500

In [9]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [11]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-0)/ results_per_page)
n_pages

725

In [12]:
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 

  0%|          | 0/725 [00:00<?, ?it/s]

In [16]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    # add a 200ms pause
    time.sleep(.2)

  0%|          | 0/725 [00:00<?, ?it/s]

HTTPError: 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?location=NY%2CNY&term=Burger&offset=1000

In [17]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,VVYea3NzbklOyHEzSEavWw,7th-street-burger-times-square-new-york,7th Street Burger - Times Square,https://s3-media1.fl.yelpcdn.com/bphoto/un0Pg-...,False,https://www.yelp.com/biz/7th-street-burger-tim...,125,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",4.5,"{'latitude': 40.75238, 'longitude': -73.98939}","[delivery, pickup]",$$,"{'address1': '485 7th Ave', 'address2': None, ...",,,5229.695192
1,S5ItalpL6lIosnBfWJm7uw,stack-d-burger-brooklyn,Stack’d Burger,https://s3-media1.fl.yelpcdn.com/bphoto/nBS4Ls...,False,https://www.yelp.com/biz/stack-d-burger-brookl...,6,"[{'alias': 'chickenshop', 'title': 'Chicken Sh...",5.0,"{'latitude': 40.68823, 'longitude': -73.98132}",[],NaN,"{'address1': '20 Nevins St', 'address2': '', '...",,,2202.348453
2,8xpGj_YkI2X6QxqXUJPw7g,7th-street-burger-new-york-18,7th Street Burger,https://s3-media2.fl.yelpcdn.com/bphoto/GYqi52...,False,https://www.yelp.com/biz/7th-street-burger-new...,334,"[{'alias': 'burgers', 'title': 'Burgers'}]",4.5,"{'latitude': 40.72697, 'longitude': -73.98546}","[delivery, pickup]",$$,"{'address1': '91 E 7th St', 'address2': None, ...",+16464906797,(646) 490-6797,2506.692303
3,XRjeE9yFANgsG12FWUOoXg,gair-brooklyn,Gair,https://s3-media4.fl.yelpcdn.com/bphoto/ZLAi5s...,False,https://www.yelp.com/biz/gair-brooklyn?adjust_...,2,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",5.0,"{'latitude': 40.703113254983855, 'longitude': ...",[],NaN,"{'address1': '41 Washington St', 'address2': '...",+17182332595,(718) 233-2595,486.167974
4,H26qMBb0OFNn2xmSTM1Phw,kings-of-kobe-new-york-8,Kings of Kobe,https://s3-media2.fl.yelpcdn.com/bphoto/AF-AC5...,False,https://www.yelp.com/biz/kings-of-kobe-new-yor...,451,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",4.5,"{'latitude': 40.761172, 'longitude': -74.000307}","[delivery, pickup]",$$,"{'address1': '650 W 42nd St', 'address2': None...",+16463705121,(646) 370-5121,6217.251058


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,nEe-yQHNVXLmKj_GLLcElw,sheikh-shack-jamaica-2,Sheikh Shack,https://s3-media1.fl.yelpcdn.com/bphoto/Udc5SK...,False,https://www.yelp.com/biz/sheikh-shack-jamaica-...,41,"[{'alias': 'halal', 'title': 'Halal'}, {'alias...",4.5,"{'latitude': 40.71826657788154, 'longitude': -...","[delivery, pickup]",$$,"{'address1': '82-37 164th St', 'address2': '',...",+13478293052,(347) 829-3052,16149.421298
996,aGmh7n-HXgcIPs5xFhK2Tw,bosna-express-ridgewood,Bosna Express,https://s3-media3.fl.yelpcdn.com/bphoto/mLjai1...,False,https://www.yelp.com/biz/bosna-express-ridgewo...,69,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",4.0,"{'latitude': 40.7047891351813, 'longitude': -7...","[delivery, pickup]",$,"{'address1': '791 Fairview Ave', 'address2': '...",+17184977577,(718) 497-7577,7746.034165
997,pGZ4AOzVlhveJi6WJ1Xz6g,bareburger-park-slope-brooklyn,Bareburger - Park Slope,https://s3-media3.fl.yelpcdn.com/bphoto/7TBH3P...,False,https://www.yelp.com/biz/bareburger-park-slope...,533,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",3.5,"{'latitude': 40.67188, 'longitude': -73.97761}","[delivery, pickup]",$$,"{'address1': '170 7th Ave', 'address2': '', 'a...",+17187682273,(718) 768-2273,3984.887073
998,KifkRh0gGD0voBw4FHFCVA,chirping-chicken-new-york-2,Chirping Chicken,https://s3-media1.fl.yelpcdn.com/bphoto/N5pZAL...,False,https://www.yelp.com/biz/chirping-chicken-new-...,660,"[{'alias': 'tradamerican', 'title': 'American'...",3.5,"{'latitude': 40.7816, 'longitude': -73.97905}","[delivery, pickup]",$$,"{'address1': '355 Amsterdam Ave', 'address2': ...",+12127876631,(212) 787-6631,8565.309536
999,KYqmJsGwTCOPWY2_--8aOg,elsie-lane-new-hyde-park-2,Elsie Lane,https://s3-media3.fl.yelpcdn.com/bphoto/mxM8vF...,False,https://www.yelp.com/biz/elsie-lane-new-hyde-p...,168,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.0,"{'latitude': 40.73118, 'longitude': -73.69297}","[delivery, pickup]",$$,"{'address1': '29 Jericho Turnpike', 'address2'...",+15163028412,(516) 302-8412,25555.991477


In [19]:
## Drop duplicate ids and confirm there are no more duplicates
final_df = final_df.drop_duplicates(subset='id')
final_df.duplicated(subset='id').sum()

0

In [21]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_NY_Burger.csv.gz', compression='gzip',index=False)